In [95]:
import datetime
import pandas as pd 
import numpy as np 
import os
import glob 
import matplotlib.pyplot as plt
import itertools

## Strategy

In [96]:
def compute_returns(prices): 
    return (prices - prices.shift(1)) / prices

def compute_moving_average(returns, window): 
    return returns.rolling(window).mean()

def compute_signal(returns, signal_function, *args): 
    signal = signal_function(returns, *args)
    return signal

def compute_position(signal, position_scale = 0.1): 
    return signal * position_scale

def compute_strategy_returns(returns, positions): 
    return 1 + (returns * positions.shift(1)).sum(axis = 1)

def compute_strategy(prices, signal_function, *args): 
    returns = compute_returns(prices)
    signal = compute_signal(returns, signal_function, *args)
    positions = compute_position(signal)
    strategy_returns = compute_strategy_returns(returns, positions)

    all_days = strategy_returns.index

    level = 100
    all_levels = {all_days[0]: level}
    for day in all_days[1:]: 
        level *= strategy_returns[day]
        all_levels[day] = level
    return pd.Series(all_levels)

## Signal Options

In [ ]:
def return_signal(returns): 
    signal = returns.map(lambda x: 1 if x > 0 else (0 if x == 0 else -1))
    return signal

def average_return_signal(returns, window): 
    average_returns = compute_moving_average(returns, window)
    signal = average_returns.map(lambda x: 1 if x > 0 else (0 if x == 0 else -1))
    return signal

def crossing_averages_signal(returns, short_window, long_window): 
    short_ma = compute_moving_average(returns, short_window)
    long_ma = compute_moving_average(returns, long_window)
    difference = short_ma - long_ma
    signal = difference.map(lambda x: 1 if x > 0 else (0 if x == 0 else -1))
    return signal

def selective_return_signal(returns, window, num_chosen): 
    average_returns = compute_moving_average(returns, window)
    ranked_returns = average_returns.rank(axis = 1, ascending = False)
    signal = ranked_returns.map(lambda x: 1 if x < num_chosen + 1 else -1)
    return signal

def selective_return_long_only_signal(returns, window, num_chosen): 
    average_returns = compute_moving_average(returns, window)
    ranked_returns = average_returns.rank(axis = 1, ascending = False)
    signal = ranked_returns.map(lambda x: 1 if x < num_chosen + 1 else 0)
    return signal

## Analytics

In [97]:
def sharpe_ratio(levels: pd.Series): 
    returns = np.log(levels).diff()
    volatility = returns.std() * np.sqrt(252)
    return returns.mean() * 252 / volatility

def max_drawdown(levels: pd.Series): 
    levels_list = levels.tolist()
    min_idx = np.argmax(np.maximum.accumulate(levels_list) - levels_list) 
    max_idx = np.argmanx(levels_list[:min_idx])
    min_val = levels_list[min_idx]
    max_val = levels_list[max_idx]
    return (max_val - min_val) / max_val * 100
    

In [99]:
def test_parameters_for_strategy(prices, signal_function, *args): 
    records = {}
    max_sharpe_ratio = -10 # Initialise to low value
    max_key = None
    parameter_combinations = list(itertools.product(*args))
    for combination in parameter_combinations: 
        levels = compute_strategy(prices, signal_function, *combination)
        sharpe = sharpe_ratio(levels)
        mdd = max_drawdown(levels)
        records[combination] = [sharpe, mdd]
        if sharpe > max_sharpe_ratio: 
            max_sharpe_ratio = sharpe
            max_key = combination
    return records, max_sharpe_ratio, max_key

### Your testing here ... 

In [100]:
prices = pd.read_pickle(r"input_prices.pkl")
prices.head()

,AAPL,AMZN,GOOGL,MSFT
2004-08-19,0.4626,1.9269,2.5099,16.9629
2004-08-20,0.4639,1.9708,2.7094,17.0129
2004-08-23,0.4681,1.9678,2.7367,17.0879
2004-08-24,0.4812,1.9478,2.6233,17.0879
2004-08-25,0.4978,2.0102,2.6516,17.2824
